In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

cuda:1


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/2D Klein Gordon/stan'

In [4]:
# !pip install smt

In [5]:
def true_2D_1(xt): #True function for 2D_1 Klein Gordon Equation x \in [-50,50] , t \in [0,10]
    y = xt[:,0]*np.cos(xt[:,1])
    return y.reshape(-1,1)    

In [6]:
loss_thresh = 0.01
label = "KG_rowdy"

x = np.linspace(-5,5,500).reshape(-1,1)
t = np.linspace(0,10,1000).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

y_true = true_2D_1(xt)
y_true_norm = np.linalg.norm(y_true,2)

#bound_pts_idx = ((X == -5) + (X == 5) + (T == 0)).reshape(-1,)

#xt_bound = xt[bound_pts_idx,:]
#y_bound = y_true[bound_pts_idx,:]


xt_test_tensor = torch.from_numpy(xt).float().to(device)


lb_xt = xt[0]
ub_xt = xt[-1]

In [7]:
xt.shape

(500000, 2)

In [8]:
def trainingdata(N_I,N_B,N_f,seed):
    '''Boundary Conditions''' 
    
    x01 = np.array([[0.0,1.0],[0.0,0.0]])
    sampling = LHS(xlimits=x01,random_state=seed)
    samples = sampling(N_I)
    xt_BC1 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC1 = true_2D_1(xt_BC1)
    

    x01 = np.array([[0.0,0.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state=seed+1)
    samples = sampling(int(N_B/2))
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    xt_BC2 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC2 = true_2D_1(xt_BC2)
    
    x01 = np.array([[1.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state=seed+2)
    samples = sampling(int(N_B/2))
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    xt_BC3 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC3 = true_2D_1(xt_BC3)

    xt_BC = np.vstack((xt_BC1,xt_BC2,xt_BC3))
    y_BC = np.vstack((y_BC1,y_BC2,y_BC3))

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll, xt_BC)) # append training points to collocation points 

    return xt_coll, xt_BC, y_BC

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val,rowdy_terms):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            
        
        self.omega1 = Parameter(torch.ones((len(layers)-2,1))) 
        self.omega1.requiresGrad = True
        
        
        self.alpha = Parameter(torch.zeros(rowdy_terms,len(layers)-2))
        self.alpha.requiresGrad = True
        
        
        self.omega = Parameter((1/n_val)*torch.ones(rowdy_terms,len(layers)-2))
        self.omega.requiresGrad = True
        
        self.n = torch.tensor(n_val)

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.omega1[i,0]*z)
            for j in range(rowdy_terms):
                a = a + self.alpha[j,i]*self.n*torch.sin((j+1)*self.n*self.omega[j,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC(self,xt,y):
                
        loss_bc = self.loss_function(self.forward(xt), y)
                
        return loss_bc
    
    def loss_PDE(self, xt_coll, f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        y = self.forward(g) 
        
        
        y_x_t = autograd.grad(y,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        y_xx_tt = autograd.grad(y_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        #du_dx = u_x_t[:,[0]]
        
        d2y_dx2 = y_xx_tt[:,[0]]
        d2y_dt2 = y_xx_tt[:,[1]]    
        

        f = d2y_dt2 - d2y_dx2 + torch.pow(y,2) + (g[:,0]*torch.cos(g[:,1])).reshape(-1,1) - (torch.pow(g[:,0],2)*torch.pow(torch.cos(g[:,1]),2)).reshape(-1,1)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_BC,y_BC,xt_coll,f_hat):

        loss_BC = self.loss_BC(xt_BC,y_BC)
        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_val = loss_BC + loss_f
        
        return loss_val
         
    'test neural network'
    def test(self):
        y_pred1 = self.forward(xt_test_tensor[:250000])
        y_pred1 = y_pred1.cpu().detach().numpy()
        
        y_pred2 = self.forward(xt_test_tensor[250000:])
          
        y_pred2 = y_pred2.cpu().detach().numpy()
        
        y_pred = np.vstack((y_pred1.reshape(-1,1),y_pred2.reshape(-1,1)))
   
        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re  

In [10]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    omega_val.append(PINN.omega.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [11]:
def train_step(xt_BC, y_BC, xt_coll, f_hat,seed):
    # x_coll_np_array = colloc_pts(N_f,seed*123)
    # x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    # f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_BC, y_BC, xt_coll,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        return loss

    optimizer.step(closure)

In [12]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0

    xt_coll, xt_BC, y_BC = trainingdata(N_I,N_B,N_f,rep*11)
    xt_coll = torch.from_numpy(xt_coll).float().to(device)
    xt_BC = torch.from_numpy(xt_BC).float().to(device)
    y_BC = torch.from_numpy(y_BC).float().to(device)

    f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
 
    for i in range(max_iter):
        train_step(xt_BC, y_BC, xt_coll,f_hat,i)

        loss_np = PINN.loss(xt_BC, y_BC, xt_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
        
   
            
        

    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))
    
  

In [13]:
max_reps = 10 #10
max_iter = 200 #200


train_loss_full = []
test_mse_full = []
test_re_full = []
alpha_full = []
omega_full = []
elapsed_time= np.zeros((max_reps,1))
time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

n_val = 3
rowdy_terms = 2

N_I = 200  #Total number of data points for 'y'
N_B = 400
N_f = 10000 #Total number of collocation points

for reps in range(max_reps):
    print(reps)
    print(label)

    train_loss = []
    test_mse_loss = []
    test_re_loss = []
    alpha_val = []
    omega_val = []

    torch.manual_seed(reps*36)

    layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

    PINN = Sequentialmodel(layers,n_val,rowdy_terms)

    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())


    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.05, 
                            max_iter = 20, 
                            max_eval = 30, 
                            tolerance_grad = 1e-8, 
                            tolerance_change = 1e-8, 
                            history_size = 100, 
                            line_search_fn = 'strong_wolfe')




    nan_flag = train_model(max_iter,reps)

    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    #elapsed_time[reps] = time.time() - start_time
    alpha_full.append(alpha_val)
    omega_full.append(omega_val)



  #print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "alpha": alpha_full,  "omega": omega_full,"label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

0
KG_rowdy
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 41.311394 Test MSE 9.291133415797278 Test RE 1.4569424876214552
1 Train Loss 34.419327 Test MSE 9.100978031060254 Test RE 1.4419562815984348
2 Train Loss 28.744106 Test MSE 8.93486506829422 Test RE 1.4287362355586426
3 Train Loss 21.769268 Test MSE 9.316529091746462 Test RE 1

96 Train Loss 0.26564506 Test MSE 3.466015841630826 Test RE 0.8898633739976585
97 Train Loss 0.2643405 Test MSE 3.476417195250743 Test RE 0.891197593228605
98 Train Loss 0.26293993 Test MSE 3.479070859029169 Test RE 0.8915376685853617
99 Train Loss 0.26135677 Test MSE 3.4814657106764186 Test RE 0.8918444650315832
100 Train Loss 0.26019266 Test MSE 3.48595130482088 Test RE 0.8924188157368663
101 Train Loss 0.25899714 Test MSE 3.4874994647210014 Test RE 0.8926169616000931
102 Train Loss 0.25785795 Test MSE 3.49029464754619 Test RE 0.8929746000590297
103 Train Loss 0.2568129 Test MSE 3.491365460322459 Test RE 0.8931115706271172
104 Train Loss 0.25578958 Test MSE 3.4960955992828 Test RE 0.8937163643659967
105 Train Loss 0.25477517 Test MSE 3.4950443028813503 Test RE 0.8935819813929671
106 Train Loss 0.2538118 Test MSE 3.4929725828581586 Test RE 0.8933171026187997
107 Train Loss 0.2529637 Test MSE 3.493549100912632 Test RE 0.893390820945628
108 Train Loss 0.25197828 Test MSE 3.4896968913030

199 Train Loss 0.21148294 Test MSE 3.615507699247158 Test RE 0.9088510339628102
Training time: 294.71
1
KG_rowdy
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
1
0 Train Loss 55.323456 Test MSE 8.701219596743089 Test RE 1.4099318619310415
1 Train Loss 44.01976 Test MSE 8.4133903023919 Test RE 1.3864160537442158
2 Train Loss 40.11522 Test MSE 9.180

95 Train Loss 0.68112695 Test MSE 6.38169374130739 Test RE 1.207469138542414
96 Train Loss 0.67869544 Test MSE 6.395409750139041 Test RE 1.2087660331212107
97 Train Loss 0.6768583 Test MSE 6.394555158433842 Test RE 1.208685269262677
98 Train Loss 0.67457044 Test MSE 6.4030029912250725 Test RE 1.2094834014757874
99 Train Loss 0.6721691 Test MSE 6.4044579002965305 Test RE 1.2096208048493275
100 Train Loss 0.66997766 Test MSE 6.422531293867012 Test RE 1.2113263786326554
101 Train Loss 0.66856956 Test MSE 6.426191939081213 Test RE 1.211671538838451
102 Train Loss 0.666751 Test MSE 6.438541920473449 Test RE 1.2128352870127272
103 Train Loss 0.6651076 Test MSE 6.444200532099871 Test RE 1.2133681294402279
104 Train Loss 0.66318536 Test MSE 6.457919848338217 Test RE 1.2146590367082395
105 Train Loss 0.6614183 Test MSE 6.45798720495841 Test RE 1.214665371187142
106 Train Loss 0.65948707 Test MSE 6.478250834947614 Test RE 1.2165695443638491
107 Train Loss 0.65710974 Test MSE 6.507863445532953 Te

199 Train Loss 0.5269486 Test MSE 7.047063371764815 Test RE 1.268855448132649
Training time: 289.01
2
KG_rowdy
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
2
0 Train Loss 53.5143 Test MSE 9.04279995483546 Test RE 1.4373400337801059
1 Train Loss 36.188377 Test MSE 6.999661031594093 Test RE 1.2645807452662063
2 Train Loss 29.089884 Test MSE 6.3343

...

146 Train Loss 7.811537e-05 Test MSE 2.8011617973695663e-05 Test RE 0.0025297473966675046
147 Train Loss 7.7468314e-05 Test MSE 2.7872302890163577e-05 Test RE 0.0025234487375502147
148 Train Loss 7.68641e-05 Test MSE 2.6501749251870497e-05 Test RE 0.0024606244244827777
149 Train Loss 7.608475e-05 Test MSE 2.5748063838040497e-05 Test RE 0.0024253831100317447
150 Train Loss 7.48593e-05 Test MSE 2.4946532665041396e-05 Test RE 0.0023873338498508335
151 Train Loss 7.396743e-05 Test MSE 2.4127157438646892e-05 Test RE 0.0023478002220348702
152 Train Loss 7.2592884e-05 Test MSE 2.5181477144948394e-05 Test RE 0.002398549366110095
153 Train Loss 7.1564304e-05 Test MSE 2.7094121401229928e-05 Test RE 0.002487972619050127
154 Train Loss 7.0834176e-05 Test MSE 2.728395551150366e-05 Test RE 0.002496673356282195
155 Train Loss 7.030756e-05 Test MSE 2.6726637052527986e-05 Test RE 0.00247104251864996
156 Train Loss 6.981757e-05 Test MSE 2.669734756696394e-05 Test RE 0.0024696881506491536
157 Train Loss 

34 Train Loss 0.57587814 Test MSE 2.7233799174309326 Test RE 0.7887914156628624
35 Train Loss 0.5659981 Test MSE 2.7185976041775763 Test RE 0.7880985440753254
36 Train Loss 0.5557211 Test MSE 2.7303142073531372 Test RE 0.7897949904858362
37 Train Loss 0.5440046 Test MSE 2.749357211186172 Test RE 0.7925444790281463
38 Train Loss 0.528408 Test MSE 2.7985426307003145 Test RE 0.7996022799379336
39 Train Loss 0.51887894 Test MSE 2.8121865097658927 Test RE 0.8015490810830727
40 Train Loss 0.5113238 Test MSE 2.833016016222614 Test RE 0.8045120904038588
41 Train Loss 0.50467056 Test MSE 2.8564381850046594 Test RE 0.8078309263541283
42 Train Loss 0.49803782 Test MSE 2.8862671059619727 Test RE 0.8120379388865488
43 Train Loss 0.4915623 Test MSE 2.8768929110793917 Test RE 0.8107181731627062
44 Train Loss 0.48544776 Test MSE 2.8904628182667302 Test RE 0.8126279466968209
45 Train Loss 0.48076677 Test MSE 2.911550599494314 Test RE 0.8155868808630113
46 Train Loss 0.47384775 Test MSE 2.93881562633794

138 Train Loss 0.24353912 Test MSE 3.4639904281568565 Test RE 0.8896033341229626
139 Train Loss 0.2430673 Test MSE 3.460453623304487 Test RE 0.8891490665683768
140 Train Loss 0.24254364 Test MSE 3.4638224089746723 Test RE 0.8895817589713694
141 Train Loss 0.24204361 Test MSE 3.462671841101255 Test RE 0.8894340018000743
142 Train Loss 0.24156645 Test MSE 3.4637471559893003 Test RE 0.8895720956516623
143 Train Loss 0.24118356 Test MSE 3.4641635705569427 Test RE 0.8896255665937102
144 Train Loss 0.24059516 Test MSE 3.4646779265866146 Test RE 0.8896916095620688
145 Train Loss 0.24008955 Test MSE 3.467339397978615 Test RE 0.8900332624161501
146 Train Loss 0.23967801 Test MSE 3.4639585318075223 Test RE 0.8895992383893538
147 Train Loss 0.239292 Test MSE 3.4613337349066664 Test RE 0.8892621298750607
148 Train Loss 0.23882672 Test MSE 3.46380548511306 Test RE 0.8895795857685902
149 Train Loss 0.23852418 Test MSE 3.462881786707686 Test RE 0.889460965072814
150 Train Loss 0.23824191 Test MSE 3.4

33 Train Loss 1.025771 Test MSE 5.825278336718275 Test RE 1.153629626946931
34 Train Loss 0.97462547 Test MSE 5.931738151108312 Test RE 1.1641234721794973
35 Train Loss 0.93706477 Test MSE 5.960078193293308 Test RE 1.1669010725820559
36 Train Loss 0.91151977 Test MSE 5.986000246801649 Test RE 1.1694359095940539
37 Train Loss 0.8902232 Test MSE 6.047379795965742 Test RE 1.1754162286217666
38 Train Loss 0.87210584 Test MSE 6.049407720072467 Test RE 1.1756132937298038
39 Train Loss 0.8616776 Test MSE 6.029886140469647 Test RE 1.1737148952087684
40 Train Loss 0.84788126 Test MSE 6.071827627092344 Test RE 1.1777897683814997
41 Train Loss 0.8336705 Test MSE 6.092417254618738 Test RE 1.1797850267035974
42 Train Loss 0.82130724 Test MSE 6.102768873897145 Test RE 1.1807868870226703
43 Train Loss 0.8103733 Test MSE 6.116099320585286 Test RE 1.1820757962695045
44 Train Loss 0.8015362 Test MSE 6.134537769765564 Test RE 1.1838562809395532
45 Train Loss 0.7918036 Test MSE 6.143969997813518 Test RE 1

138 Train Loss 0.47356966 Test MSE 7.153094824289097 Test RE 1.2783655289756681
139 Train Loss 0.47263974 Test MSE 7.154021795113392 Test RE 1.278448358099873
140 Train Loss 0.47173974 Test MSE 7.161974938930596 Test RE 1.279158787817297
141 Train Loss 0.47069687 Test MSE 7.164879905176628 Test RE 1.2794181810996974
142 Train Loss 0.4698813 Test MSE 7.1711364927044725 Test RE 1.2799766723233688
143 Train Loss 0.4690097 Test MSE 7.177954382289397 Test RE 1.2805849905726023
144 Train Loss 0.46828994 Test MSE 7.18208262186876 Test RE 1.2809531875175777
145 Train Loss 0.46731687 Test MSE 7.1896086096385465 Test RE 1.2816241568128428
146 Train Loss 0.466362 Test MSE 7.197451693715715 Test RE 1.2823230228205376
147 Train Loss 0.46545848 Test MSE 7.205660582860543 Test RE 1.2830540765514664
148 Train Loss 0.4647621 Test MSE 7.211067189879022 Test RE 1.2835353418243356
149 Train Loss 0.4640267 Test MSE 7.222019996573915 Test RE 1.2845097454675414
150 Train Loss 0.46335357 Test MSE 7.2258361940

In [14]:
import scipy.io as sio
import numpy as np

In [15]:
for tune_reps in range(75):
    label = "KG_rowdy_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

FileNotFoundError: [Errno 2] No such file or directory: 'KG_rowdy_tune0.mat'

In [ ]:
lrnr_tune[1]